# My hangman file

In [1]:
import json
import numpy
import math
import requests
import random
import string
import secrets
import time
import re
import collections

In [2]:
tf = open("words_250000_train.txt","r")
full_dict = tf.read().splitlines()
tf.close()


In [3]:
random.choice(full_dict)

'successlessly'

So, I think the best way to do this would be to make a decision tree from scratch, but I think there is an issue of outliers that I am not super sure how to deal with. I think the dictionary is big enough that there are some words that the algorithm should figure out how to throw out. Like if there are 2 moves left, it should make the highest probability assumption it can which eliminates trees with more than 2 moves of surprise.

So... I think how the algorithm should work is, it doesnt just pick the ONE letter that would maximize the entropy next, it should find like a PATH of guesses that maximally reduces the entropy.

I am assuming there are always 6 tries, which seems a bit low...

It seems a bit complex though... If I build a tree I don't have to constantly re-calculate it, but I just move from node to node.

HOW DO I MAKE A DECISION TREE CLASSIFIER WITH NO LABELS THOUGH?

For testing purposes, I want to just remake hangman here so I can mess with it. So gut all of the connection to some external anything. I think I know how it works from running it a bit.

In [4]:
from nltk import ngrams, FreqDist

In [38]:
class MyHangman(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.guessed_letters = []
        #should include a self word and self correct word
        self.word = ""
        self.correct_word = ""
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        self.current_dictionary = []
        self.alphabet = list(string.ascii_lowercase)
        self.vowels = list(["a", "e", "i", "o", "u"])
        self.guess_list = []
        #self.init_grams()

    def init_grams(self):
        #set up n-grams for current dictionary
        grams2 = []
        grams3 = []
        grams4 = []
        grams5 = []
        for word in self.full_dictionary:
            #is the word clean? It should be...
            #when I loaded the data it's fine...
            #the API has be a bit nervous so lets just put this here
            word = word.replace(" ", "") #remove any padding that may mess with this

            grams2.extend(list(ngrams(word, 2, pad_left = True, pad_right = True)))
            grams3.extend(list(ngrams(word, 3, pad_left = True, pad_right = True)))
            grams4.extend(list(ngrams(word, 4, pad_left = True, pad_right = True)))
            grams5.extend(list(ngrams(word, 5, pad_left = True, pad_right = True)))
        
        #Ok, so now need the frequency distributions for these
        freq_dist_2 = FreqDist(grams2)
        freq_dist_3 = FreqDist(grams3)
        freq_dist_4 = FreqDist(grams4)
        freq_dist_5 = FreqDist(grams5)

        #need to actually extract the data from these distributions to use it
        self.freq_dist_2 = [(item, freq_dist_2.get(item)) for item in freq_dist_2]
        self.freq_dist_3 = [(item, freq_dist_3.get(item)) for item in freq_dist_3]
        self.freq_dist_4 = [(item, freq_dist_4.get(item)) for item in freq_dist_4]
        self.freq_dist_5 = [(item, freq_dist_5.get(item)) for item in freq_dist_5]

        self.words_by_length = {}
        min_word_length = 3 #shortest in the full_dict at least
        max_word_length = 40 #I guess this is pretty arbitrary

        for i in range(min_word_length, max_word_length):
            self.words_by_length.update({i : []})

        #I do it in this kinda roundabout way because maybe there are words of 25 and 28 but no 27 or whatever
        #for word in self.full_dictionary:
        #    self.full_dictionary[len(word)].append(word)
        
            
        
    def generate_correct_word(self):
        self.correct_word = random.choice(self.full_dictionary)


    #I think this is an easier way to apply ngrams... I don't really know as much about the subject
        #so there is an equal or greater likelihood that this is completely wrong
    def find_potential_guesses(self, freq_N, input_word, guess_list):
        #This takes as input a freq dist for n-grams and a word
        #it outputs all the n-grams from freq dist that match along with the 
        potential_guesses = []
        filtered_dist = []
        if(freq_N == []):
            dimension = 0
        else:
            dimension = len(freq_N[0][0])
        word_ngrams = list(ngrams(input_word, dimension))
        #print(dimension)

        for fe in freq_N:
            #Ok..... so here is the idea...
            #if the ngram in fe matches, then it must be a match 
            #to one of the n-gons of the target word with wildcards
            fe_gram = fe[0]
            occurrences = fe[1]
            fe_gram = ["_" if x == None else x for x in fe_gram] #convert None to underscore if they exist
            #now, try to match it to one of the n-grams of the word...
            matched_letters_count = 0 #you must match at least one letter to be considered

            #possibilities, letters match, one is empty or both are empty
            #print(fe_gram[0])


            #for these guesses to be valid, they MUST guess something NOT IN THE GUESS LIST!
            #so, need to filter that out at the start.
            fe_letters = []
            for l in fe_gram:
                if l != "_":
                    fe_letters.append(l)
            #print(fe_letters)
            if fe_letters == []:
                is_subset = False
            else:
                is_subset = set(fe_letters).issubset(guess_list)

            ##also have to throw it out if guess is a vowel and ratio is too high?
            #vowels = list(("a", "e", "i", "o", "u"))
            #if fe_letters == []:
            #    vowelCondition = False
            #else:
            #    vowelCondition = set(fe_letters).issubset(vowels)
            vowelCondition = True #fix later

            remove_set = set(guess_list)
            remove_set.add("_")
            
            if(is_subset == False and vowelCondition == True):
                #print("I am inside the if statement")
                for word_gram in word_ngrams:
                    matched_letters_count = 0
                    for i in range(dimension):
                        if(word_gram[i] != "_"): #there is a letter here, does the fe match?
                            if(fe_gram[i] == word_gram[i]):
                                matched_letters_count += 1
                                #print(word_gram)
                                #print(fe_gram)
                            #elif(fe_gram[i] == "_"): #word has a letter and fe_gram has a blank so its fine
                            #    continue
                            else:
                                #this happens if we have a direct contradiction, immediately reject this
                                matched_letters_count = 0
                                break
                        #we don't have to check what the corresponding letter is if word_gram[i] is blank!
                    if(matched_letters_count != 0):
                        potential_guesses.append((set(fe_gram) - remove_set, dimension ,matched_letters_count,occurrences))
                        filtered_dist.append(fe)
                        #print(set(fe_gram).issubset(guess_list))
                        
        

        return potential_guesses
        
    def evaluate_potential_guesses(self, potential_guesses):
        #each potential guess is in the form:
        #({letters}, dimension from ngram, matched letters count, occurrences of pattern in dict)

        #So, which do we pick:
        #prioritize ngram length first
        #then pick based on most frequent letter
        #I guess how many letters match isnt as important...
        #I have read that vowel ratio may make a difference... maybe implement later

        sorted_guesses = sorted(potential_guesses, key = lambda tup: tup[1], reverse = True)
        if(sorted_guesses != []):
            max_dim = sorted_guesses[0][1]
            #print(max_dim)
            max_dim_guesses = [x for x in sorted_guesses if x[1] == max_dim]
            max_dim_guesses.sort(key = lambda tup: tup[3], reverse = True)

            print(max_dim_guesses[0])
            max_dim_guesses[0][0]
            max_guess = max_dim_guesses[0]
            print(max_guess)
            if len(max_guess) > 0:
                max_guess = list(max_guess)[0]
            return list(max_guess) #this is a set in general
        else:
            return "!"
    
    def update_grams(self, word, guess_list):
        #update ngrams to remove items associated with incorrect letters
        word = word.replace("_", "")
        incorrect_letters = set(guess_list) - set(word)
        self.freq_dist_2 = [(item[0], item[1]) for item in self.freq_dist_2 if set(item[0]).isdisjoint(set(incorrect_letters))]
        self.freq_dist_3 = [(item[0], item[1]) for item in self.freq_dist_3 if set(item[0]).isdisjoint(set(incorrect_letters))]
        self.freq_dist_4 = [(item[0], item[1]) for item in self.freq_dist_4 if set(item[0]).isdisjoint(set(incorrect_letters))]
        self.freq_dist_5 = [(item[0], item[1]) for item in self.freq_dist_5 if set(item[0]).isdisjoint(set(incorrect_letters))]



    def guess(self, word): # word input example: "_ p p _ e "
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################

        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        # The extra space characters must be coming from the website...
        clean_word = word.replace(" ", "") #I am putting this here in case I forget later to remove the space stuff when I paste this into the solution
        #clean_word = word.replace("_",".") #I am using underscores

        # find length of passed word
        len_word = len(clean_word)
        
        # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
        current_dictionary = self.current_dictionary
        new_dictionary = []
        #print(current_dictionary[0])
        
        # iterate through all of the words in the old plausible dictionary
        for dict_word in current_dictionary:
            # continue if the word is not of the appropriate length
            if len(dict_word) != len_word:
                continue
                
            # if dictionary word is a possible match then add it to the current dictionary
            if re.match(clean_word,dict_word):
                new_dictionary.append(dict_word)
        
        # overwrite old possible words dictionary with updated version
        self.current_dictionary = new_dictionary
        
        
        # count occurrence of all characters in possible word matches
        full_dict_string = "".join(new_dictionary)
        
        c = collections.Counter(full_dict_string)
        sorted_letter_count = c.most_common()
        #print(sorted_letter_count)                
        
        guess_letter = '!'

        #print(self.freq_dist_2[0:10])
        #print(self.freq_dist_3[0:10])
        #print(self.freq_dist_4[0:10])
        #print(self.freq_dist_5[0:10])
        #print(self.guessed_letters)
        #print(len(self.guessed_letters))

        freq_2_guesses = self.find_potential_guesses(self.freq_dist_2, word, self.guessed_letters)
        freq_3_guesses = self.find_potential_guesses(self.freq_dist_3, word, self.guessed_letters)
        freq_4_guesses = self.find_potential_guesses(self.freq_dist_4, word, self.guessed_letters)
        freq_5_guesses = self.find_potential_guesses(self.freq_dist_5, word, self.guessed_letters)
        #print("freq 2 guesses ",freq_2_guesses[0:10])
        #print(freq_3_guesses[0:10])
        #print(freq_4_guesses[0:10])
        #print(freq_5_guesses[0:10])
        #print()

        

        all_guesses = freq_2_guesses + freq_3_guesses + freq_4_guesses + freq_5_guesses
        max_guess = self.evaluate_potential_guesses(all_guesses)
        
        
        # return most frequently occurring letter in all possible words that hasn't been guessed yet
        for letter,instance_count in sorted_letter_count:
            if letter not in self.guessed_letters:
                guess_letter = letter
                break
            
        # if no word matches in training dictionary, default back to ordering of full dictionary
        if guess_letter == '!':
            sorted_letter_count = self.full_dictionary_common_letter_sorted
            for letter,instance_count in sorted_letter_count:
                if letter not in self.guessed_letters:
                    guess_letter = letter
                    break            
        
        print(sorted_letter_count)  
        #print(max_guess)
        #print(sorted_letter_count(max_guess[0])) 
        if(max_guess != "!"):
            #max_guess is in a form like {"a", "b"} or something
            #let's pick the most likely letter from among these
            guess_letter = max_guess[0]
        



        return guess_letter

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

    #what do I need to replace the responses?
    #get a word
    #get remaining tries
    #print number of tries remaining
    #go through the process of resolving a guess
    #

    def handle_guess(self, guessLetter, currentWord, ans):
        #I have to convert the strings to lists, find indices of the letter (if present) and then smush the lists back into strings and spit it out
        indices = [i for i, x in enumerate(list(ans)) if x == guessLetter]
        wordList = list(currentWord)
        for i in indices:
            wordList[i] = guessLetter #now I replace the occurrences of the letter
        word = "".join(wordList)
        return word
        
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
        self.init_grams()

        #pick a word from the dictionary at random.
        #correctWord = random.choice(self.full_dictionary) #this is the initial word, but need to update it based on guesses
        self.generate_correct_word()
        word = "_" * len(self.correct_word) #I should make this a member of self right?

        tries_remains = 6 #I think this needs to just be a magic number?
        if verbose:
            print("Successfully start a new game! # of tries remaining: {0}. Word: {1}.".format(tries_remains, word))
        while tries_remains >0:
            #tries_remains -=1 tries remains decreases after WRONG answer

            #get guessed letter
            guess_letter = self.guess(word)

            #append guessed letter to guessed letters field in hangman object
            self.guessed_letters.append(guess_letter)

            self.update_grams(word, self.guess_list)
            if verbose:
                print("Guessing letter: {0}".format(guess_letter))

            #apply guessed letter to the word
            new_word = self.handle_guess(guess_letter, word, self.correct_word) 

            #check if we have the word
            if new_word == self.correct_word:
                word = new_word
                print("Success! the word was: %s" % word)
                break
            
            if new_word == word:
                #this means that we did not get a correct guess
                #decrease number of tries
                print("Incorrect guess {0}, # of tries remaining: {1}. Word: {2}.".format(guess_letter,tries_remains, word))
                tries_remains -=1
            else:
                #We have a correct letter guessed, but not the complete word
                #don't decrement tries_remains
                word = new_word
                print("Got a Letter, {0}, # of tries remaining: {1}. Word: {2}.".format(guess_letter,tries_remains, word))
        if tries_remains == 0:
            print("You Lose, the answer was: %s" % self.correct_word)
                



In [39]:
game = MyHangman()
game.start_game()

Successfully start a new game! # of tries remaining: 6. Word: _______.
[('e', 233745), ('i', 184746), ('a', 179837), ('n', 152259), ('o', 150052), ('r', 149228), ('s', 148462), ('t', 137277), ('l', 122431), ('c', 89367), ('u', 77304), ('d', 74856), ('p', 65785), ('m', 62191), ('h', 58051), ('g', 51850), ('y', 40985), ('b', 39840), ('f', 26431), ('v', 21057), ('k', 18685), ('w', 17732), ('z', 8749), ('x', 6050), ('q', 3986), ('j', 3790)]
Guessing letter: e
Got a Letter, e, # of tries remaining: 6. Word: __ee___.
({'d'}, 5, 2, 298)
({'d'}, 5, 2, 298)
[('e', 233745), ('i', 184746), ('a', 179837), ('n', 152259), ('o', 150052), ('r', 149228), ('s', 148462), ('t', 137277), ('l', 122431), ('c', 89367), ('u', 77304), ('d', 74856), ('p', 65785), ('m', 62191), ('h', 58051), ('g', 51850), ('y', 40985), ('b', 39840), ('f', 26431), ('v', 21057), ('k', 18685), ('w', 17732), ('z', 8749), ('x', 6050), ('q', 3986), ('j', 3790)]
Guessing letter: d
Incorrect guess d, # of tries remaining: 6. Word: __ee__

In [26]:
temp = ({'d', '_'}, 5, 1, 17572)

In [28]:
temp[0].remove("_")